* Filter:
    - Importance: High => Low
    - Medium questions only

## Debounce

* main pitfall is being able to access `this` from inside the callback
* there are 2 ways of doing this:
    1. have a variable initialized with the `this` value inside the returned debounced function
        * then you use `.call(thisArgs, ...args)`
    2. use an arrow function inside the setTimeout
        * the `this` inside the setTimeout will be the same one in the outer function
        * so you skip the step of having to initialize and use the outer function's `this` value
* we should NOT return an arrow function inside the debounce()
    - reason being, it's bound to the place it was created and not where it's called
* for example:

```
const increment = debounce(function(num) {
    console.log(this);
    this.val += num;
}, 10);

const obj = {
    val: 2,
    increment,
};

obj.increment(5);
```
* if you tried this in jsfiddle in which the debounce returned an arrow function instead of a function(), `this` would be the window object and not obj

In [ ]:
// my solution

/**
 * @param {Function} func
 * @param {number} wait
 * @return {Function}
 * 
 * must wait until after x milliseconds after
 * last debounced function was last called
 * in order to actually execute the real function
 */
export default function debounce(func, wait) {
  // need to settimeout before calling the function
  let timer;

  // every time we call debounced function
  // it returns a function

  return function(...args) {
    // if it's called, we clear the timer
    clearTimeout(timer);
    const that = this;
    // set a new timer
    timer = setTimeout(function() {
      func.call(that, ...args);
    }, wait);
  }
}

In [ ]:
// actual solution

/**
 * @callback func
 * @param {number} wait
 * @return {Function}
 */
export default function debounce(func, wait = 0) {
  let timeoutID = null;
  return function (...args) {
    clearTimeout(timeoutID);

    timeoutID = setTimeout(() => {
      timeoutID = null; // Not strictly necessary but good to include.
      // Has the same `this` as the outer function's
      // as it's within an arrow function.
      func.apply(this, args);
    }, wait);
  };
}

## Debounce II

* __creating static methods is exactly the same as creating methods on an object__
    - for this question, we could just use dot notation and assign the methods onto the function itself
    - remember that pretty much everything in JS is an object
* flush does not have access to the `this` value or parameters from the returned function
    - instead, you must create a function-level variables and assign them the `this` and the parameters from inside the returned function
* if a debounced function has already been invoked, calling flush() should not do anything!
    - to implement this, we can check if timer is null and if it is, we can return immediately
    - we should also set timer to null whenever we cancel or finish invoking the timer from the setTimeout

In [ ]:
// my solution

/**
 * @param {Function} func
 * @param {number} wait
 * @return {Function}
 * 
 * has to return an object with 2 methods:
 * 1) cancel => cancels pending invocations (probably just cleartimeout)
 * 2) flush => immediately invokes delayed invocations
 *  - also clears timeout but calls the function
 * 
 * pitfall: how to call `this` inside function?
 * probably just use a variable again
 */
export default function debounce(func, wait) {
  let timer = null;
  let context = null;
  let params = [];

  const myDebounce = function(...args) {
    context = this;
    params = args;
    clearTimeout(timer);
    timer = setTimeout(() => {
      timer = null;
      func.call(this, ...args);
    }, wait);
  };

  myDebounce.cancel = () => {
    clearTimeout(timer);
    timer = null;
  }

  myDebounce.flush = () => {
    if (timer === null || context === null) return;
    myDebounce.cancel();
    func.call(context, ...params);
  }

  return myDebounce;
}

In [ ]:
// actual solution

/**
 * @param {Function} func
 * @param {number} wait
 * @return {Function}
 */
export default function debounce(func, wait = 0) {
  let timeoutId = null;
  let context = undefined;
  let argsToInvoke = undefined;

  function clearTimer() {
    clearTimeout(timeoutId);
    timeoutId = null;
  }

  function invoke() {
    // Don't invoke if there's no pending callback.
    if (timeoutId == null) {
      return;
    }

    clearTimer();
    func.apply(context, argsToInvoke);
  }

  function fn(...args) {
    clearTimer();
    argsToInvoke = args;
    context = this;
    timeoutId = setTimeout(function () {
      invoke();
    }, wait);
  }

  fn.cancel = clearTimer;
  fn.flush = invoke;
  return fn;
}


## Promise.all

* just remember to have a counter for the number of items that we've processed so far
    - if it matches # of items in the iterable, then that means nothing rejected and we can return the entire result array

In [ ]:
/**
 * @param {Array} iterable
 * @return {Promise<Array>}
 */
export default function promiseAll(iterable) {
  return new Promise((resolve, reject) => {
    if (iterable.length === 0) resolve([]);
    const result = new Array(iterable.length);
    let counter = 0;

    iterable.forEach(async (item, index) => {
      try {
        const value = await item;
        result[index] = value;
        counter++;
        if (counter === iterable.length) {
          resolve(result);
        }
      } catch(e) {
        reject(e);
      }
    });

  });
}

## Throttle

* remember to check the boolean's state
    - if you canCall = false, then we know we're currently throttling the function calls
* use an anonymous function `function(){}` instead of an arrow function because we can use the `this` value if the function is used as a method
    - then you can just pass in the `this` value into `func.apply()` or `func.call()` to bind `this` to it
    - if you needed `this` in the setTimeout, you would just create a variable and assign `this` to it or use an arrow function as the callback for the setTimeout since it takes the `this` value of the place where it was created

In [ ]:
// my solution

/**
 * @callback func
 * @param {number} wait
 * @return {Function}
 * 
 * invoked IMMEDIATELY
 * has to wait X milliseconds before being able to be called again
 * 
 * should return a function with a throttled invocation of the callback
 */
export default function throttle(func, wait) {
  // keeps track of whether we can call or not
  let canCall = true;
  return function(...args) {
    // have to make sure that the function can call `this`?
    // make the call
    if (!canCall) return;

    func.apply(this, args);
    canCall = false;

    setTimeout(() => {
      canCall = true;
    }, wait);
  }
}

In [ ]:
// actual solution

/**
 * @callback func
 * @param {number} wait
 * @return {Function}
 */
export default function throttle(func, wait = 0) {
  let shouldThrottle = false;

  return function (...args) {
    if (shouldThrottle) {
      return;
    }

    shouldThrottle = true;
    setTimeout(function () {
      shouldThrottle = false;
    }, wait);

    func.apply(this, args);
  };
}